<a href="https://colab.research.google.com/github/noahfavreau/nasa-space-apps-2025/blob/main/model_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, cv, Pool

from pytorch_tabnet.tab_model import TabNetClassifier

import lightgbm as lgb

from sklearn.linear_model import LogisticRegressionCV

import xgboost as xgb

from xgboost import plot_importance

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, KFold

import random

import numpy as np

In [ ]:
n_splits = 10
kf = KFold(n_splits=n_splits, shuffle=True, random_state=random.randint(0, 100))

for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
  X_train, X_val = X.iloc[train_index], X.iloc[test_index]
  y_train, y_val = y.iloc[train_index], y.iloc[test_index]

  train_catboost(X_train, X_val, y_train, y_val)
  train_xgb(X_train, X_val, y_train, y_val)
  train_lgb(X_train, X_val, y_train, y_val)

In [ ]:
def train_catboost(X_train, X_val, y_train, y_val):

  catboost_fold_accuracies = []
  catboost_oof_preds = np.zeros(len(X))

  catboost_model = CatBoostClassifier(
                           iterations=200,
                           task_type="GPU",
                           devices='0',
                           depth=6,
                           loss_function='MultiClass',
                           verbose=0,
                           eval_metric='AUC',
                           random_state=random.randint(0, 100)
                          )

  catboost_train_pool = Pool(X_train,
                            y_train,
                             )

  catboost_val_pool = Pool(X_val,
                            y_val,
                             )

  catboost_model.fit(catboost_train_pool,
                     eval_set=catboost_val_pool,
                     early_stopping_rounds=20,
                     use_best_model=True)

  catboost_val_preds = catboost_model.predict(X_val)
  catboost_oof_preds[test_index] = catboost_val_preds

  catboost_accuracy = accuracy_score(y_val, catboost_val_preds)
  catboost_report = classification_report(y_val, catboost_val_preds)

  catboost_mean_accuracy = np.mean(catboost_fold_accuracies)
  catboost_std_accuracy = np.std(catboost_fold_accuracies)

In [ ]:
def train_lgb(X_train, X_val, y_train, y_val):

  lgb_fold_accuracies = []
  lgb_oof_preds = np.zeros(len(X))

  params = {
    'objective' : 'multiclass',
    'num_class' : 3,
    'metric' : "multi_logloss",
    'verbose' : 0
  }

  lgb_train_data = lgb.Dataset(X_train, label=y_train)
  lgb_val_data = lgb.Dataset(X_val, label=y_val, reference=lgb_train_data)

  lgb_model = lgb.train(params,
                      lgb_train_data,
                      200,
                      early_stopping_rounds=20,
                      valid_sets=[lgb_val_data])

  lgb_val_preds = lgb_model.predict(X_test,
                           num_iteration=lgb_model.best_iteration)

lgb_accuracy = accuracy_score(y_test, lgb_y_pred)
lgb_report = classification_report(y_test, lgb_y_pred)

In [ ]:
xgb_model = XGBClassifier(num_classes=,
                          objective='multi:softmax'
                          eval_metrics='mlogloss',
                          use_label_encoder=False,
                          max_depth=4,
                          n_estimators=100,
                          random_state=random.randint(0, 100))

xgb_model.fit(X_train, y_train)

xgb_y_pred = xgb_model.predict(X_test)

y_prob = xgb_model.predict_proba(X_test)

xgb_accuracy = accuracy_score(y_test, xgb_y_pred)
xgb_report = classification_report(y_test, xgb_y_pred)